# DESAFIO - Aula_23: Matriz_Custo e Curva_Lucro

Base de Dados <br>
Link: https://raw.githubusercontent.com/madmashup/targeted-marketing-predictive-engine/master/banking.csv <br>
Resumo: O conjunto de dados está relacionado a campanhas de marketing direto (chamadas telefônicas) de uma instituição bancária portuguesa. A variável resposta (y) é binária e indica se o cliente subscreveu um depósito a prazo (1-Sim, 0-Não). <br>
Objetivo: Ajustar um modelo de regressão logística, em uma base de treinamento, para a resposta binária, fazer a previsão desta resposta e avaliar a qualidade de ajuste do modelo em uma base de teste.


## Importando os Dados

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

data = pd.____('______/banking.csv',header=0)
#drop na
data = data.dropna()
lista = ('age','education','contact','month','day_of_week','duration',
        'campaign','pdays','previous','emp_var_rate','cons_price_idx','cons_conf_idx', 
        'euribor3m','nr_employed')
#exclui algumas variáveis da análise
for col in lista:
    data.drop(col, axis=1, inplace=True)
print(data.shape)

O conjunto de dados fornece a informação dos clientes do banco. Inclui 41,188 registros e 16 variáveis.

In [ ]:
data.head()

# Parte_1: Tratando os dados

- ## Var resposta

Transformando a variável Income-Renda em 0's and 1's

In [ ]:
# grafico de barras para a variável dependente
sns.____(x='y',data=data, palette='hls')
plt.show()

A proporção de 1's é de aproximandamente 14%

- ## Verificando valores missing

In [ ]:
data.isnull().sum()

Não há dados missing na base de análise

- ## job: trabalho do cliente

Tem muitas linhas missing, então vamos colocá-las em uma nova classe, substituindo por 0 e ver como fica o gráfico

In [ ]:
# grafico de barras para a variável customer job 
sns.countplot(y='job',data=data)
plt.show()
#há muitas categorias: vamos agrupar as classes que tem menos obs

In [ ]:
# unindo as categorias de trabalho, para depois não criar muitas dummys
def agrupa(job):
    if job in ['blue-collar']:
        return 'blue_collar'
    elif job in ['technician']:
        return 'technician'
    elif job in ['admin.']:
        return 'admin' 
    elif job in ['management']:
        return 'management'
    elif job in ['services']:
        return 'services'
    
    else: 
        return 'other'    

In [ ]:
data['job'] = data['job'].apply(agrupa)

In [ ]:
# grafico de barras para a variável customer job 
sns.countplot(y='job',data=data)
plt.show()

- ## Marital: estado civil

In [ ]:
sns.countplot(x='marital',data=data)
plt.show()

A categoria unknown tem poucas obs - vamos juntá-la com a cat divorced

In [ ]:
def agrupa1(marital):
    if marital in ['unknown']:
        return 'divorced'
    else: 
        return marital   

In [ ]:
data['marital'] = data['marital'].apply(____)

In [ ]:
sns.countplot(x='marital',data=data)
plt.show()

- ## Default - crédito em default

In [ ]:
sns.countplot(x='default',data=data)
plt.show()

A categoria yes tem poucas obs - vamos juntá-la com a cat unknown

In [ ]:
def agrupa2(default):
    if default in ['yes']:
        return 'unknown'
    else: 
        return default   

In [ ]:
data['default'] = data['default'].____(agrupa2)

In [ ]:
sns.countplot(x='default',data=data)
plt.show()

- ## Criando Dummys

Crie variáveis dummy, que são variáveis com apenas dois valores, zero e um. <br>
Nos modelos de regressão logística, a codificação de todas as variáveis independentes-categoricas como variáveis dummy permite uma fácil interpretação e cálculo da odds ratios e aumenta a estabilidade e a importância dos coeficientes.

In [ ]:
data.dtypes

In [ ]:
#list of columns with dtype: object
#axes[1] coluna
#axes[0] linha
categorical_features = data.select_dtypes(include=['object']).axes[1] # retorna as vars que são do tipo objeto

# unique: retorna os valores únicos
# nunique: retorna o número de valores únicos
for col in categorical_features:
    print (col, data[col].nunique()) # retorna as vars que são do tipo objeto e a quantidade de categorias em cada uma delas

In [ ]:
#criando dummys para as variáveis categoricas
for col in categorical_features:
    data = pd.concat([data, pd.get_dummies(data[col], prefix=col, prefix_sep='_')], axis=1)
    data.drop(col, axis=1, inplace=True)

In [ ]:
data.columns

In [ ]:
data.head()

# Parte_2: Dividindo a base em train e test

In [ ]:
X = data.iloc[:,1:]
y = data.iloc[:,0]
X_train, X_test, y_train, y_test = ____________(X, y, test_size=_______, random_state=__________)

In [ ]:
print(X_train.shape, X_test.shape)

# Parte_3: Ajustar, Prever e Validar um modelo de Regressão Logística usando sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import statsmodels.api as sm
import pylab as pl


In [ ]:
X1_train = X_train[['poutcome_success' , 'default_no' , 'job_other' , 'poutcome_failure' , 'marital_single' , 'job_blue_collar' ,
       'job_services' , 'job_technician' , 'marital_divorced']]
X1_test = X_test[['poutcome_success' , 'default_no' , 'job_other' , 'poutcome_failure' , 'marital_single' , 'job_blue_collar' ,
       'job_services' , 'job_technician' , 'marital_divorced']]

############################################
# Ajustando um modelo de Regressão Logística: qual comando devo usar
model = _________()
model.fit(X1_train, y_train)
#fazendo a previsão (0 ou 1) do modelo
p_test1 = model.____________(X1_test)

In [ ]:
# Calculando confusion matrix, AUC, precision e recall para base de treinamento

# matriz de confusão
cnf_matrix1 = ______________(y_test, p_test1)
print ("\n\n ---Base de Teste---")
print ("\n\n ---Matriz de Confusão---")
print(cnf_matrix1)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X1_test, y_test)))
#0,8986112459939788

print ("\n\n ---Logistic Model---")
# Calculando a área abaixo da curva ROC
logit_roc_auc2 = _________________(y_test, model.predict(X1_test))
print ("Logistic AUC = %2.2f" % logit_roc_auc2)
print(classification_report(y_test, p_test1))

In [ ]:
# Plotando o curva ROC para a base de teste
fpr2, _________, thresholds = ____________(y_test, model.predict_proba(X1_test)[:,1])
plt.plot(__________, tpr2, label='Test(area = %0.2f)' % logit_roc_auc2)


plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Graph')
plt.legend(loc="lower right")
plt.show()

## Matriz de Custo e Curva de Lucro



O conjunto de dados está relacionado a campanhas de marketing direto (chamadas telefônicas) de uma instituição bancária portuguesa. A variável resposta (y) é binária e indica se o cliente subscreveu um depósito a prazo (1-Sim, 0-Não). 
O custo da ligação direta para o cliente é de 10 reais, mas se o cliente subescreve ao depósito temos uma beneficio de 100-10=90

In [ ]:
###### Matriz de Custo
cost_TP = 90; cost_FP = -10; cost_FN = 0; cost_TN = 0;
cb_matrix = np.array([[cost_TN, cost_FP],[cost_FN, cost_TP]])
cb_matrix

## Calculando o lucro

In [ ]:
# Calcula a probabilidade (0 a 1) para a base de teste
predict_prob = model.____________(X1_test)[:,1]
# Cria um data frame com a probabilidade predita e a y_real
df = pd.DataFrame( {"predict_prob":predict_prob, "true_value":y_test} )
# Ordena o data frame df pela prob predita
df.sort_values(by="predict_prob", inplace=True)

In [ ]:
# Defini um ponto de corte para o cálculo do lucro
p = 0.1
label_p = df.predict_prob >= p    
###### Calcula a matriz de confusão
counf = __________________(df.true_value, label_p)
lucro = (counf * cb_matrix).sum()
print(lucro)

In [ ]:
# e se eu definir outro ponto de corte
p = 0.2
label_p = df.predict_prob >= p    
###### Calcula a matriz de confusão
counf = ____________(df.true_value, label_p)
lucro = (counf * cb_matrix).sum()
print(lucro)

### Como podemos fazer para encontrar o ponto de corte ótimo, ou seja, o que retorno o maior lucro

In [ ]:
#Calcula o lucro para "cada" ponto de corte 
predict_prob = model.predict_proba(X1_test)[:,1]
df = pd.DataFrame( {"predict_prob":predict_prob, "true_value":y_test} )
df.sort_values(by="predict_prob", inplace=True)

def profit_gain(predict_prob, true_value, p_step):
    lucro = []
    p=[0]
    while p[-1]+p_step<=1:
        label_p = (df.predict_prob >= p[-1]).astype(int)    
        counf = confusion_matrix(df.true_value, label_p)
        lucro.append( (counf * cb_matrix).sum() )
        p.append(p[-1]+p_step)
    return lucro, p

# Agora vamos tornar o processo mais automatico

In [ ]:
# Carregando pacotes necessários
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GBC

In [ ]:
# ajustando alguns modelos
# Regressão Logistica
model_1 = ____()
model_1.____(X_train, y_train)

#Random Forest
model_2 = ____()
model_2.____(X_train, y_train)

#Gradient Boosting
model_3 = ____()
model_3.____(X_train, y_train)

models = model_1, model_2, model_3

In [ ]:
#Calcula o locro para "cada" ponto de corte e depois plota o lucro versus a probabilidade
def profit_gain(predict_prob, true_value, p_step):
    df = pd.DataFrame( {"predict_prob":predict_prob, "true_value":true_value} )
    ### ordenando os valores pela probabilidade predita
    df.____________(by="predict_prob", inplace=True)
    lucro = []
    p=[0]
    while p[-1]<=1:
        label_p = (df.predict_prob >= p[-1]).astype(int)    
        #calcula a matriz de confusão
        counf = ___________(df.true_value, label_p)
        lucro.append( (counf * cb_matrix).sum() )
        p.append(p[-1]+p_step)
    return lucro, p

def plot_profit( predict_prob, true_value, p_step ):
    array_lucro, p = profit_gain( ___________, __________, __________ )
    plt.plot(p[:-1], array_lucro)
    return None

def multi_plot( models, X, y_true, p_step):
    for i in models:
        predict_prob = i.predict_proba( X )[:,1]
        plot_profit( __________, y_true, p_step )
    plt.legend([i.__class__.__name__ for i in models]) # não substituir nada nesta linha
    plt.show()

In [ ]:
multi_plot( models=models, X=______, y_true=_________, p_step=________)

In [ ]:
# Plotando curva ROC para todos os modelos
#calcula a curva ROC
fpr1, tpr1, thresholds1 = ______(y_test, model_1.predict_proba(X_test)[:,1])
#calcula a área abaixo da curva ROC
logit_roc_auc1 = ______(y_test, model_1.predict(X_test))
plt.plot(______, ______, label='Test_LR(area = %0.2f)' % logit_roc_auc1)
#
fpr2, tpr2, thresholds2 = ______(y_test, model_2.predict_proba(X_test)[:,1])
logit_roc_auc2 = ______(y_test, model_2.predict(X_test))
plt.plot(______, ______, label='Test_RF(area = %0.2f)' % logit_roc_auc2)
#
fpr3, tpr3, thresholds3 = ______(y_test, model_3.predict_proba(X_test)[:,1])
logit_roc_auc3 = ______(y_test, model_3.predict(X_test))
plt.plot(______, ______, label='Test_GBC(area = %0.2f)' % logit_roc_auc3)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Graph')
plt.legend(loc="lower right")
plt.show()

Referências <br>
- https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8